# 自定义评价函数

[![下载Notebook](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_customized_metric.ipynb)&emsp;[![下载样例代码](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorial/zh_cn/mindspore_customized_metric.py)&emsp;[![查看源文件](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/intermediate/build_net/customized_metric.ipynb)

当训练任务结束，常常需要评价函数(Metrics)来评估模型的好坏。不同的训练任务往往需要不同的Metrics函数。例如，对于二分类问题，常用的评价指标有precision(准确率)、recall(召回率)等， 而对于多分类任务，可使用宏平均
(Macro)和微平均(Micro)来评估。MindSpore提供了大部分常见任务的评价函数，如`nn.Accuracy`、`nn.Pecision`、`nn.MAE`、`nn.TopKCategoricalAccuracy`和`nn.MSE`等，详情可参考：[评估指标](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.nn.html#id16)。由于MindSpore提供的评价函数无法满足所有任务的需求，很多情况下用户需要针对具体的任务自定义Metrics来评估训练的模型。本章主要介绍如何自定义Metrics以及如何在`nn.Model`中使用Metrics。

## 自定义Metrics

自定义Metrics函数需要继承`nn.Metric`父类，并重新实现父类中的`clear`方法、`update`方法和`eval`方法。下面以简单的平均绝对误差`MAE`为例，MAE公式如式（1）所示，介绍这三个函数及其使用方法。

$$ MAE=\frac{1}{n}\sum_{i=1}^n\lvert ypred_i - y_i \rvert \tag1$$

- `clear`：初始化相关的内部参数。
- `update`：接收网络预测输出和标签，计算误差，并更新内部评估结果。如果用户评估网络有多个输出，但只用两个输出进行评估，此时可以使用`set_indexes`方法重排`update`的输入用于计算评估指标。`update`一般在每个step进行计算并更新统计值。
- `eval`：计算最终评估结果，一般在一个epoch结束后计算最终的评估结果。

In [14]:
import numpy as np
import mindspore
from mindspore import Tensor
import mindspore.nn as nn


class MAE(nn.Metric):
    def __init__(self):
        super(MAE, self).__init__()
        self.clear()

    def clear(self):
        """初始化变量_abs_error_sum和_samples_num"""
        self._abs_error_sum = 0  # 保存误差和
        self._samples_num = 0  # 累计数据量

    @nn.rearrange_inputs
    def update(self, *inputs):
        """更新_abs_error_sum和_samples_num"""
        if len(inputs) != 2:
            raise ValueError('Mean absolute error need 2 inputs (y_pred, y), but got {}'.format(len(inputs)))
        # 将Tensor转换为NumPy，便于后续计算
        y_pred = inputs[0].asnumpy()
        y = inputs[1].asnumpy()
        # 计算预测值与真实值的绝对误差
        abs_error_sum = np.abs(y.reshape(y_pred.shape) - y_pred)
        self._abs_error_sum += abs_error_sum.sum()
        # 样本的总数
        self._samples_num += y.shape[0]

    def eval(self):
        """计算最终评估结果"""
        if self._samples_num == 0:
            raise RuntimeError('Total samples num must not be 0.')
        return self._abs_error_sum / self._samples_num


# 网络有两个输出：x，y
x = Tensor(np.array([[0.1, 0.2, 0.6, 0.9], [0.1, 0.2, 0.6, 0.9]]), mindspore.float32)
y = Tensor(np.array([[0.1, 0.25, 0.7, 0.9], [0.1, 0.25, 0.7, 0.9]]), mindspore.float32)
error = MAE()
error.clear()
error.update(x, y)
result = error.eval()
print("output(x,y):", result)

# 网络有三个输出:x，y，z
x = Tensor(np.array([[0.1, 0.2, 0.6, 0.9], [0.1, 0.2, 0.6, 0.9]]), mindspore.float32)
y = Tensor(np.array([[0.1, 0.25, 0.7, 0.9], [0.1, 0.25, 0.7, 0.9]]), mindspore.float32)
z = Tensor(np.array([[0.1, 0.25, 0.7, 0.8], [0.1, 0.25, 0.7, 0.8]]), mindspore.float32)
# 设置使用x,z进行评估
error = MAE().set_indexes([0, 2])
error.clear()
error.update(x, y, z)
result = error.eval()
print("output(x,z):", result)

output(x,y): 0.1499999612569809
output(x,z): 0.24999992549419403


>使用`set_indexes`方法，需要用装饰器`mindspore.nn.rearrange_inputs`修饰`update`方法，否则使用set_indexes配置的输入不生效。

## 在Model中使用Metrics

[mindspore.Model](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore/mindspore.Model.html#mindspore.Model)是用于训练和评估的高层API，可以将自定义或MindSpore已有的Metrics作为参数传入，Model能够自动调用传入的Metrics进行评估。如下示例在`Model`中传入上述自定义的Metrics函数，使用验证数据集进行验证并打印验证结果。

In [ ]:
import numpy as np
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Model
from mindspore import dataset as ds
from mindspore.nn import LossBase
from mindspore.common.initializer import Normal


# 生成数据及对应标签
def get_data(num, w=2.0, b=3.0):
    for _ in range(num):
        x = np.random.uniform(-10.0, 10.0)
        noise = np.random.normal(0, 1)
        y = x * w + b + noise
        yield np.array([x]).astype(np.float32), np.array([y]).astype(np.float32)


# 加载数据集
def create_dataset(num_data, batch_size=16):
    dataset = ds.GeneratorDataset(list(get_data(num_data)), column_names=['data', 'label'])
    dataset = dataset.batch(batch_size)
    return dataset


# 定义线性回归网络
class LinearNet(nn.Cell):
    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc = nn.Dense(1, 1, Normal(0.02), Normal(0.02))

    def construct(self, x):
        return self.fc(x)


# 通过继承LossBase来定义损失函数
class L1Loss(LossBase):
    def __init__(self, reduction="mean"):
        """初始化，求loss均值"""
        super(L1Loss, self).__init__(reduction)
        # 求绝对值
        self.abs = ops.Abs()

    def construct(self, base, target):
        x = self.abs(base - target)
        # 返回loss均值
        return self.get_loss(x)


# 创建数据集
ds_train = create_dataset(num_data=160)
# 定义网络
net = LinearNet()
# 定义损失函数
loss = L1Loss()
# 定义优化器
opt = nn.Momentum(net.trainable_params(), learning_rate=0.005, momentum=0.9)
# 定义model，将自定义metrics函数MAE传入Model中
model = Model(net, loss, opt, metrics={"MAE": MAE()})
# 模型训练
ds_eval = create_dataset(num_data=100)
model.train(epoch=1, train_dataset=ds_train, dataset_sink_mode=False)

# 创建评估数据集
ds_eval = create_dataset(num_data=100)
# 模型评估
output = model.eval(ds_eval)
# 打印结果
print(output)

```Python
{'MAE': 5.212465476989746}
```